# Step 0 Extract Useful Data

In [1]:
import pandas as pd

In [2]:
#Read the database
raw_data=pd.read_csv("loan.csv")

/home/ubuntu/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
raw_data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
raw_data.sample(50).to_csv("sample.csv")

In [4]:
#record the label to use

#categorical data: home_ownership,purpose,loan_status

target=['loan_amnt','term','int_rate','home_ownership','annual_inc','purpose','earliest_cr_line','inq_last_6mths','loan_status']

#extract the useful info

raw_data[target].to_csv("data.csv")

# Clean data and convert categorical data

In [1]:
import pandas as pd
data=pd.read_csv("data.csv",index_col=0)

In [2]:
data.head()

,loan_amnt,term,int_rate,home_ownership,annual_inc,purpose,earliest_cr_line,inq_last_6mths,loan_status
0,5000.0,36 months,10.65,RENT,24000.0,credit_card,Jan-1985,1.0,Fully Paid
1,2500.0,60 months,15.27,RENT,30000.0,car,Apr-1999,5.0,Charged Off
2,2400.0,36 months,15.96,RENT,12252.0,small_business,Nov-2001,2.0,Fully Paid
3,10000.0,36 months,13.49,RENT,49200.0,other,Feb-1996,1.0,Fully Paid
4,3000.0,60 months,12.69,RENT,80000.0,other,Jan-1996,0.0,Current


In [3]:
#Assign default characters to default

data['loan_status'].drop_duplicates().iloc[7]

'Does not meet the credit policy. Status:Fully Paid'

In [4]:
data['binary_default']=data.loan_status.apply(lambda x: 1 if ((x=="Fully Paid") or (x=="Current")or(x=="Issued")or x==('Does not meet the credit policy. Status:Fully Paid')) else 0)

In [5]:
data.purpose.drop_duplicates().values

array(['credit_card', 'car', 'small_business', 'other', 'wedding',
       'debt_consolidation', 'home_improvement', 'major_purchase',
       'medical', 'moving', 'vacation', 'house', 'renewable_energy',
       'educational'], dtype=object)

In [6]:
#Assign credit line purpose binary status
purpose=dict(zip(data.purpose.drop_duplicates().values,range(len(data.purpose.drop_duplicates().values))))

purpose

{'car': 1,
 'credit_card': 0,
 'debt_consolidation': 5,
 'educational': 13,
 'home_improvement': 6,
 'house': 11,
 'major_purchase': 7,
 'medical': 8,
 'moving': 9,
 'other': 3,
 'renewable_energy': 12,
 'small_business': 2,
 'vacation': 10,
 'wedding': 4}

In [7]:
data['purpose_binary']=data.purpose.apply(lambda x: purpose[x])

In [8]:
#Adjust homeownership
data.home_ownership.value_counts()

MORTGAGE    443557
RENT        356117
OWN          87470
OTHER          182
NONE            50
ANY              3
Name: home_ownership, dtype: int64

In [9]:
home_ownership={"MORTGAGE":0,"RENT":1,"OWN":2}

data['home_own_binary']=data.home_ownership.apply(lambda x: home_ownership[x] if x in home_ownership.keys() else 0)

In [10]:
#comvert term to time

data.term.value_counts()

 36 months    621125
 60 months    266254
Name: term, dtype: int64

In [11]:
data['term_digit']=data.term.apply(lambda x: int(x[0:2]))

In [12]:
data['earliest_cre_line']=pd.to_datetime(data.earliest_cr_line,format='%b-%Y')

In [13]:
#Use earliest time in this column as the baseline time, taking all other time as the differences
baseline_date=data['earliest_cre_line'].min()

import numpy as np

#Adjust datetime to baseline date
data['earliest_cre_line']=data['earliest_cre_line']-baseline_date
data['earliest_cre_line']=data['earliest_cre_line'].apply(lambda x: x.days)

In [14]:
data['earliest_cre_line']

0         14976.0
1         20179.0
2         21124.0
3         19024.0
4         18993.0
5         22220.0
6         22462.0
7         23011.0
8         22006.0
9         22159.0
10        19724.0
11        16710.0
12        22006.0
13        21731.0
14        17287.0
15        23254.0
16        19997.0
17        18110.0
18        21823.0
19        20820.0
20        19663.0
21        14276.0
22        15157.0
23        21640.0
24        20971.0
25        21216.0
26        21823.0
27        21823.0
28        14823.0
29        22950.0
           ...   
887349    19783.0
887350    21397.0
887351    17014.0
887352    22980.0
887353    16437.0
887354    20971.0
887355    18049.0
887356    18597.0
887357    21366.0
887358    21670.0
887359    21670.0
887360    14457.0
887361    18536.0
887362    21793.0
887363    22919.0
887364    11627.0
887365    22950.0
887366    24503.0
887367    20514.0
887368    18383.0
887369    21428.0
887370    18475.0
887371    21336.0
887372    18779.0
887373    

In [15]:
cleaned_data=data[['loan_amnt','term_digit','int_rate','home_own_binary','annual_inc','purpose_binary','earliest_cre_line','inq_last_6mths','binary_default']]

In [16]:
cleaned_data.to_csv("cleaned_data.csv")